### Metrics (RMSE, MAE, Coverage, Width, Winkler)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Set Path
MCR_PATH   = "/Users/jinnanut/Desktop/Code/MCR_summary_weekly.xlsx"
HYB_PATH   = "/Users/jinnanut/Desktop/Code/MCR+ML_summary_weekly.xlsx"
RAW_PATH   = "/Users/jinnanut/Library/Mobile Documents/com~apple~CloudDocs/NUT/Exeter University/Master/Term 3/BEMM466 Business Project/Dissertation/Data/WeeklyData.xlsx"
OUTPUT_DIR = Path("/Users/jinnanut/Desktop/Code/model_eval_outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

MAKE_PLOTS = True
ALPHA = 0.20          # P10–P90 is an 80% nominal interval
W_ANCHOR = "W-SUN"    # use the same weekly anchor 

# Set function for transform date and set metrics
def _prep(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.str.strip().str.lower()
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"])
    return df

def metrics(y, yhat, low, up, alpha=0.20):
    y = np.asarray(y, float); yhat = np.asarray(yhat, float)
    low = np.asarray(low, float); up = np.asarray(up, float)
    err = yhat - y
    rmse = float(np.sqrt(np.mean(err**2)))
    mae  = float(np.mean(np.abs(err)))
    coverage = float(np.mean((y >= low) & (y <= up)))
    width = float(np.mean(up - low))
    penalty = (2/alpha)*((low-y)*(y<low) + (y-up)*(y>up))  # Winkler (lower=better)
    winkler = float(np.mean((up - low) + penalty))
    return rmse, mae, coverage, width, winkler

# Load forecasts
df_mcr = _prep(pd.read_excel(MCR_PATH))
df_hyb = _prep(pd.read_excel(HYB_PATH))

# Required columns in each forecast file: date, mean, p10, p90
for name, df in [("MCR", df_mcr), ("MCR+ML", df_hyb)]:
    need = {"date", "mean", "p10", "p90"}
    miss = need - set(df.columns)
    if miss:
        raise ValueError(f"{name} is missing columns: {sorted(miss)}")

# Load ACTUALS from raw data
raw = _prep(pd.read_excel(RAW_PATH))   # columns: date, category, quantity
if not {"date", "quantity"}.issubset(raw.columns):
    raise ValueError("RAW file must have at least 'date' and 'quantity' columns (plus 'category' if multiseries).")

has_cat = "category" in raw.columns and ("category" in df_mcr.columns) and ("category" in df_hyb.columns)

# resample to weekly with the same anchor and build actuals DataFrame
raw = raw.set_index("date").sort_index()
if has_cat:
    actual_weekly = (
        raw.groupby("category")["quantity"]
            .resample(W_ANCHOR, label="right", closed="right")
            .sum()
            .reset_index()
    )
else:
    actual_weekly = (
        raw[["quantity"]]
            .resample(W_ANCHOR, label="right", closed="right").sum()
            .reset_index()
    )
actual_weekly.rename(columns={"quantity": "actual_demand", "date":"date"}, inplace=True)

# Standardize forecast columns and merge
def _std(df, suffix):
    cols = ["date", "mean", "p10", "p90"]
    if has_cat: cols = ["date", "category", "mean", "p10", "p90"]
    out = df[cols].copy()
    out = out.rename(columns={
        "mean": f"forecast_demand_{suffix}",
        "p10":  f"p10_{suffix}",
        "p90":  f"p90_{suffix}",
    })
    return out

mcr_std = _std(df_mcr, "mcr")
hyb_std = _std(df_hyb, "hyb")

keys = ["date"] + (["category"] if has_cat else [])

merged = (
    mcr_std.merge(hyb_std, on=keys, how="inner")
           .merge(actual_weekly, on=keys, how="inner")   # <-- TRUE actuals joined here
           .sort_values(keys)
           .reset_index(drop=True)
)

# Drop any rows with missing interval bounds (rare after inner merges)
merged = merged.dropna(subset=[
    "forecast_demand_mcr","forecast_demand_hyb","p10_mcr","p90_mcr","p10_hyb","p90_hyb","actual_demand"
]).copy()

print(f"[Eval] n={len(merged)} rows | {merged['date'].min().date()} → {merged['date'].max().date()} | "
      f"{'per-category' if has_cat else 'single series'}")

# overall
overall_df = eval_block(merged)
print("\n=== Overall (TEST) ===")
print(overall_df.round(3).to_string(index=False))


[Eval] n=4375 rows | 2020-02-09 → 2022-10-09 | per-category

=== Overall (TEST) ===
 Model   RMSE    MAE  Coverage_Prob  Avg_Interval_Width  Winkler_Score
   MCR 33.990 21.090          0.385              37.965        113.576
MCR+ML 20.461 12.508          0.038               0.053        124.869


### Hypothesis testing (Diebold–Mariano)

In [11]:
from scipy.stats import norm

# Set function 
def _newey_west_var(d, lag):
    """HAC variance estimator with Bartlett weights."""
    d = np.asarray(d, float)
    d = d[np.isfinite(d)]
    n = d.size
    if n < 2:
        return np.nan
    d = d - d.mean()
    gamma0 = np.dot(d, d) / n
    var = gamma0
    for k in range(1, min(lag, n-1) + 1):
        w = 1.0 - k/(lag+1.0)  # Bartlett weight
        cov = np.dot(d[k:], d[:-k]) / n
        var += 2.0 * w * cov
    return var

def diebold_mariano(y, yhat1, yhat2, loss="mse", h=1, automatic_lag=True, alternative="two-sided"):
    """
    DM test comparing forecast 1 vs 2.
      loss: 'mse' (squared error) or 'mae' (absolute error)
      h   : forecast horizon; also used as HAC lag if automatic_lag=False
      automatic_lag: if True, uses floor(1.1447*n^(1/3)) as NW lag; else uses h-1
      alternative: 'two-sided' | 'greater' (model1 worse) | 'less' (model1 better)
    Returns: (dm_stat, p_value, n_effective)
    """
    y     = np.asarray(y,     float)
    f1    = np.asarray(yhat1, float)
    f2    = np.asarray(yhat2, float)

    mask = np.isfinite(y) & np.isfinite(f1) & np.isfinite(f2)
    y, f1, f2 = y[mask], f1[mask], f2[mask]
    n = y.size
    if n < 5:
        return np.nan, np.nan, n

    e1 = y - f1
    e2 = y - f2
    if loss.lower() == "mse":
        L1, L2 = e1**2, e2**2
    elif loss.lower() == "mae":
        L1, L2 = np.abs(e1), np.abs(e2)
    else:
        raise ValueError("loss must be 'mse' or 'mae'")

    d = L1 - L2  # loss differential (positive => model1 worse)

    # HAC variance
    if automatic_lag:
        # Andrews (1991) type rule-of-thumb used widely: floor(1.1447 * n^(1/3))
        lag = int(np.floor(1.1447 * (n ** (1/3))))
    else:
        lag = max(0, h - 1)

    s2 = _newey_west_var(d, lag)
    if not np.isfinite(s2) or s2 <= 0:
        return np.nan, np.nan, n

    dm_stat = d.mean() / np.sqrt(s2 / n)

    # p-value
    if alternative == "two-sided":
        pval = 2.0 * (1.0 - norm.cdf(abs(dm_stat)))
    elif alternative == "greater":
        pval = 1.0 - norm.cdf(dm_stat)
    elif alternative == "less":
        pval = norm.cdf(dm_stat)
    else:
        raise ValueError("alternative must be 'two-sided', 'greater', or 'less'")

    return float(dm_stat), float(pval), int(n)


dm_mae = diebold_mariano(
    merged['actual_demand'],
    merged['forecast_demand_mcr'],
    merged['forecast_demand_hyb'],
    loss="mae", automatic_lag=True  
)

dm_mse = diebold_mariano(
    merged['actual_demand'],
    merged['forecast_demand_mcr'],
    merged['forecast_demand_hyb'],
    loss="mse", automatic_lag=True
)

print(f"DM (MAE): stat={dm_mae[0]:.3f}, p={dm_mae[1]:.4f}, n={dm_mae[2]}")
print(f"DM (MSE): stat={dm_mse[0]:.3f}, p={dm_mse[1]:.4f}, n={dm_mse[2]}")

# Interpretation
def interpret(dm_tuple, name):
    stat, p, n = dm_tuple
    if not np.isfinite(stat) or not np.isfinite(p):
        print(f"[{name}] DM is undefined (check data; n={n}).")
        return
    if p < 0.05:
        direction = "Model 2 (hybrid) better" if stat > 0 else "Model 1 (MCR) better"
        print(f"[{name}] p < 0.05 ⇒ reject H0 (equal accuracy). {direction}.")
    else:
        print(f"[{name}] p ≥ 0.05 ⇒ cannot reject equal predictive accuracy.")

interpret(dm_mae, "MAE")
interpret(dm_mse, "MSE")


DM (MAE): stat=7.055, p=0.0000, n=4375
DM (MSE): stat=5.180, p=0.0000, n=4375
[MAE] p < 0.05 ⇒ reject H0 (equal accuracy). Model 2 (hybrid) better.
[MSE] p < 0.05 ⇒ reject H0 (equal accuracy). Model 2 (hybrid) better.


### Demand-shock scenarios (-80%, -50%, -30%)

In [12]:
# Metrics 
def metrics(y, yhat, low, up, alpha=0.20):
    y = np.asarray(y, float); yhat = np.asarray(yhat, float)
    low = np.asarray(low, float); up = np.asarray(up, float)
    err = yhat - y
    rmse = float(np.sqrt(np.mean(err**2)))
    mae  = float(np.mean(np.abs(err)))
    coverage = float(np.mean((y >= low) & (y <= up)))
    width = float(np.mean(up - low))
    penalty = (2/alpha)*((low-y)*(y<low) + (y-up)*(y>up))  # Winkler (lower is better)
    winkler = float(np.mean((up - low) + penalty))
    return rmse, mae, coverage, width, winkler

# Evaluate a dataframe for both models
def eval_two_models(df, alpha=0.20):
    y = df["actual_demand"]
    out = {}

    # MCR
    rmse, mae, cov, wid, wkl = metrics(
        y, df["forecast_demand_mcr"], df["p10_mcr"], df["p90_mcr"], alpha
    )
    out["MCR"] = dict(RMSE=rmse, MAE=mae, Coverage=cov, Width=wid, Winkler=wkl)

    # Hybrid
    rmse, mae, cov, wid, wkl = metrics(
        y, df["forecast_demand_hyb"], df["p10_hyb"], df["p90_hyb"], alpha
    )
    out["MCR+ML"] = dict(RMSE=rmse, MAE=mae, Coverage=cov, Width=wid, Winkler=wkl)
    return out


baseline = eval_two_models(merged, alpha=ALPHA)

# scenario constructors 
def apply_uniform_drop(df, drop_pct):
    """
    drop_pct = 0.30 -> 30% drop (multiply actuals by 0.70)
    """
    g = df.copy()
    g["actual_demand"] = g["actual_demand"] * (1.0 - drop_pct)
    return g

def apply_lockdown(df, drop_pct=0.80, weeks=8, start_date=None, end_date=None):
    """
    Reduce actuals by drop_pct in a contiguous block (default center 8 weeks).
    If start_date/end_date provided, they define the lockdown window.
    """
    g = df.copy()
    idx = g["date"].sort_values().unique()
    if start_date is None or end_date is None:
        # center window
        n = len(idx)
        w = min(weeks, n)
        s = max(0, (n - w)//2)
        mask_dates = set(idx[s:s+w])
    else:
        mask_dates = set(pd.date_range(pd.to_datetime(start_date),
                                       pd.to_datetime(end_date), freq="W-SUN"))
    mask = g["date"].isin(mask_dates)
    g.loc[mask, "actual_demand"] = g.loc[mask, "actual_demand"] * (1.0 - drop_pct)
    return g

# Evaluate scenarios 
scenarios = {
    "drop_30": apply_uniform_drop(merged, 0.30),
    "drop_50": apply_uniform_drop(merged, 0.50),
    "lockdown_80": apply_lockdown(merged, drop_pct=0.80, weeks=8)  # adjust weeks/dates as needed
}

rows = []
for scen_name, scen_df in scenarios.items():
    res = eval_two_models(scen_df, alpha=ALPHA)
    for model in ["MCR", "MCR+ML"]:
        base = baseline[model]
        cur  = res[model]
        rows.append({
            "Model": model,
            "Scenario": scen_name,
            # % degradation vs baseline: positive = worse
            "RMSE_Degradation_%": 100.0 * (cur["RMSE"] / base["RMSE"] - 1.0),
            "MAE_Degradation_%":  100.0 * (cur["MAE"]  / base["MAE"]  - 1.0),
            # coverage change in percentage points
            "Coverage_Change_pp": 100.0 * (cur["Coverage"] - base["Coverage"]),
            # Winkler: larger is worse — report % change
            "Winkler_Change_%":   100.0 * (cur["Winkler"] / base["Winkler"] - 1.0),
        })

robustness_tbl = pd.DataFrame(rows)
print("\n=== Scenario Robustness (Degradation vs Base) ===")
print(robustness_tbl.to_string(index=False, float_format=lambda x: f"{x:,.1f}"))



=== Scenario Robustness (Degradation vs Base) ===
 Model    Scenario  RMSE_Degradation_%  MAE_Degradation_%  Coverage_Change_pp  Winkler_Change_%
   MCR     drop_30               -34.2              -30.9                12.1             -38.9
MCR+ML     drop_30                -3.6                4.9                 0.0               4.9
   MCR     drop_50               -47.3              -43.4                19.7             -54.3
MCR+ML     drop_50                22.3               35.9                 0.0              36.0
   MCR lockdown_80                -4.1               -6.6                 4.2              -6.7
MCR+ML lockdown_80                15.2               13.9                 0.0              13.9


/var/folders/qm/xr3xsq8s2hl_27kvzqb6mkz40000gn/T/ipykernel_80062/94727038.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[10.2 10.2 10.2 10.2 10.2 10.2 10.2 10.2 10.2 10.2 10.2 10.2 10.2 10.2
 10.2 32.6 32.6 32.6 32.6 32.6 32.6 32.6 32.6 32.6 32.6 32.6 32.6 32.6
 32.6 32.6  5.2  5.2  5.2  5.2  5.2  5.2  5.2  5.2  5.2  5.2  5.2  5.2
  5.2  5.2  5.2  7.4  7.4  7.4  7.4  7.4  7.4  7.4  7.4  7.4  7.4  7.4
  7.4  7.4  7.4  7.4 14.  14.  14.  14.  14.  14.  14.  14.  14.  14.
 14.  14.  14.  14.  14.   6.8  6.8  6.8  6.8  6.8  6.8  6.8  6.8  6.8
  6.8  6.8  6.8  6.8  6.8  6.8  0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   5.4  5.4  5.4  5.4  5.4
  5.4  5.4  5.4  5.4  5.4  5.4  5.4  5.4  5.4  5.4  9.4  9.4  9.4  9.4
  9.4  9.4  9.4  9.4  9.

### Uncertainty Sensitivity

In [13]:
# Uncertainty Sensitivity
rows = []
for model in ["MCR", "MCR+ML"]:
    if model == "MCR":
        low, up, mean = merged["p10_mcr"], merged["p90_mcr"], merged["forecast_demand_mcr"]
    else:
        low, up, mean = merged["p10_hyb"], merged["p90_hyb"], merged["forecast_demand_hyb"]
    
    width = (up - low).to_numpy()
    avg_width = np.mean(width)
    rel_width = np.mean(width / np.where(mean != 0, mean, np.nan))
    
    rows.append({
        "Model": model,
        "Avg_Width": avg_width,
        "Rel_Width_vs_Forecast": rel_width
    })

uncertainty_tbl = pd.DataFrame(rows)
print("\n=== Uncertainty Sensitivity (Spread) ===")
print(uncertainty_tbl.round(3).to_string(index=False))



=== Uncertainty Sensitivity (Spread) ===
 Model  Avg_Width  Rel_Width_vs_Forecast
   MCR     37.965                  2.404
MCR+ML      0.053                    NaN


### Error decomposition (bias/variance)

In [14]:
def bias_variance_from_df(df, y_col, yhat_col):
    """Return Bias, Var(errors), and MSE for a model given actuals and forecasts."""
    y = np.asarray(df[y_col], float)
    yhat = np.asarray(df[yhat_col], float)
    mask = np.isfinite(y) & np.isfinite(yhat)
    e = yhat[mask] - y[mask]
    if e.size == 0:
        return np.nan, np.nan, np.nan
    bias = float(e.mean())
    var_err = float(e.var(ddof=0))     # population variance over horizon
    mse = float((e**2).mean())
    # sanity: mse ≈ bias^2 + var_err (up to rounding)
    return bias, var_err, mse

# Overall bias–variance table
rows = []
b_mcr, v_mcr, mse_mcr = bias_variance_from_df(
    merged, "actual_demand", "forecast_demand_mcr"
)
rows.append({"Model":"MCR", "Bias": b_mcr, "Variance": v_mcr, "MSE": mse_mcr})

b_hyb, v_hyb, mse_hyb = bias_variance_from_df(
    merged, "actual_demand", "forecast_demand_hyb"
)
rows.append({"Model":"MCR+ML", "Bias": b_hyb, "Variance": v_hyb, "MSE": mse_hyb})

bv_table = pd.DataFrame(rows)
print("\n=== Bias–Variance ===")
print(bv_table[["Model","Bias","Variance"]].round(3).to_string(index=False))

# Breakdown per-category
if "category" in merged.columns:
    per_cat = []
    for cat, g in merged.groupby("category"):
        b, v, mse = bias_variance_from_df(g, "actual_demand", "forecast_demand_mcr")
        per_cat.append({"Category":cat, "Model":"MCR", "Bias":b, "Variance":v, "MSE":mse})
        b, v, mse = bias_variance_from_df(g, "actual_demand", "forecast_demand_hyb")
        per_cat.append({"Category":cat, "Model":"MCR+ML", "Bias":b, "Variance":v, "MSE":mse})
    per_cat_table = pd.DataFrame(per_cat)
    # Sort categories by absolute bias or variance 
    per_cat_table = per_cat_table.sort_values(["Model","Variance"], ascending=[True,False])



=== Bias–Variance ===
 Model    Bias  Variance
   MCR -14.833   935.315
MCR+ML  -1.743   415.596
